### Testing the MATH Dataset

In [1]:
%load_ext autoreload
%autoreload 2

import sys, os
sys.path.append(os.path.dirname(os.getcwd()))

import constants

from data_classes.math_data_loader import MathDataLoader, MathProblem
from llm_agents.ChainOfThoughtMathSolver import ChainOfThoughtMathSolver


In [2]:
math_data_loader = MathDataLoader()
random_problem = math_data_loader.get_random_problem(level=1)

math_solver = ChainOfThoughtMathSolver()
response = math_solver(random_problem.question)

random_problem.is_correct_answer(response, verbose=True)

Loaded 5000 math problems from 7 problem types
Level: 1 | Response: \boxed{180^\circ} | Answer: \boxed{180^{\circ}} | Is Correct: True


True